In [1]:
from osgeo import gdal

In [2]:
import os
from utils.GenUtils import make_folder, get_paths, chunk_creator, folder_file_size, question

In [3]:
os.environ["ISISROOT"]="/opt/conda/envs/isis/"
os.environ["ISISDATA"]="/isis-data"

In [4]:
import kalasiris as isis

In [5]:
config = {
            "inst":"lronac",
            "maptemplate":"/home/jovyan/scripts/PyISIS-Parallel/maptemplates/Center_EquirectangularMoon.map",
            "oxt":"tiff",
            "PATH":"/home/jovyan/Data/NAC/",
            "DSTPATH":"/home/jovyan/Data/NAC/",
            "byte":"n"
            }

In [6]:
if config['inst']=='lronac':
    from utils.KalaUtils import lro
    proc_func=lro
    cam = 'nac'
    ixt='IMG'
elif config['inst']=='lrowac':
    from utils.KalaUtils import lro
    proc_func=lro
    cam = 'wac'
    ixt='IMG'
elif config['inst']=='m3L1':
    from utils.KalaUtils import m3L1
    proc_func=m3L1
    cam = None
    ixt='LBL'
elif config['inst']=='m3L2':
    from utils.KalaUtils import m3L2
    proc_func=m3L2
    cam = None
    ixt='LBL'
else:
    print('Cannot find utils')
    
maptemplate=config['maptemplate']
byte = config['byte']

In [7]:
def mapper(src, proc_func, maptemplate, ixt, oxt, cam, byte):
    src_basename = os.path.basename(src).split('.'+ixt)[0]
    dst_basename = DSTPATH+'/'+src_basename
    print(dst_basename)
#    if os.path.isfile(dst_basename+'_lev2.JP2'):
#        print ("File exist")
#    elif os.path.isfile(dst_basename+'.JP2'):
#        print ("File exist")
#    else:
    try:
        proc_func(src, dst_basename, maptemplate, config['oxt'], cam, byte)
    except Exception as e:
        print(e)
        
  

In [8]:
def parallel_mapper(files, JOBS, proc_func, maptemplate, ixt, oxt, cam, byte):
    from joblib import Parallel, delayed, parallel_backend
    with parallel_backend("loky", inner_max_num_threads=2):

        Parallel (n_jobs=JOBS)(delayed(mapper)(files[i],proc_func, maptemplate, ixt, oxt, cam, byte)
                                for i in range(len(files)))

In [9]:
def main(PATH, DPATH, ixt, maptemplate):
    image_list = get_paths(PATH, ixt) 
    total_size, max_size, av_fsize = folder_file_size(PATH,image_list)
    from tqdm import tqdm
    import psutil
    avram=psutil.virtual_memory().total >> 30
    avcores=psutil.cpu_count(logical=False)
    avthreads=psutil.cpu_count(logical=True)
    ram_thread = avram/avthreads
    req_mem = avthreads*max_size
    if req_mem > avcores and req_mem > avram:
        JOBS = int(avcores - avcores/(avram/avcores))
    else:
        JOBS = int(avcores - avcores/(avram/avcores))
    #JOBS = 1
    with tqdm(total=len(image_list),
             desc = 'Generating Images',
             unit='File') as pbar:
        filerange = len(image_list)
        chunksize = round(filerange/JOBS)
        if chunksize <1:
            chunksize=1
            JOBS = filerange
        chunks = []
        for c in chunk_creator(image_list, JOBS):
            chunks.append(c)
        # from time import time
        from datetime import datetime
        for i in range(len(chunks)):
            start = datetime.now()
            dt_string = start.strftime("%d/%m/%Y %H:%M:%S")
            #print(f'Loop {i} started at: {dt_string}', chunks[i])
            files = chunks[i]
            parallel_mapper(files, JOBS, proc_func, maptemplate, ixt, config['oxt'], cam, byte)
            pbar.update(JOBS)

In [10]:
if __name__ == "__main__":
    PATH=config['PATH']
    DSTPATH=config['DSTPATH']
    os.makedirs(DSTPATH, exist_ok=True)
    print(ixt)
    main(PATH, DSTPATH, ixt, maptemplate)

IMG


Generating Images: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:35<00:00, 17.93s/File]


In [21]:
from kalasiris import lronac2isis, lrowac2isis, lronaccal, lrowaccal, chan1m32isis, cam2map, spiceinit, lronacecho, stretch
def RAWtoL0(inst, src, L0):
    inst(src, to=L0)    

def L0toL1(inst, dst_basename, L0, L1):
    inst(L0,to=L1)
    #os.remove(L0)

def L1toL1nacecho(L1,L1echo):
    lronacecho(L1, to=L1echo)
    #os.remove(L1)
    
def L1toL2(maptemplate, L1, L2):
    #res=GetRes(maptemplate)
    cam2map(L1, to=L2, PIXRES='MAP', map=maptemplate)
    #os.remove(L1)

    
def L2toStd(L2cub, L2std, byte):
    opts = f' -mask none'
    
    nodata=rio.open(L2cub).nodata #using rio since src_map.GetMetadata() return empty
    if byte.lower() in ['yes','y','ye']:
        nodata=0
        opts = opts+f' -ot Byte  -scale -a_nodata {nodata}'            

    try:
        src_map = gdal.Open(L2cub)
        
        gdal.Translate(L2std, src_map, options=opts)#,scaleParams=[])
        return 'Done'
        #os.remove(dst_lev2)
    except Exception as e:
        return(e)     

In [14]:
ixt = 'IMG'
oxt='tiff'
image_list = get_paths(PATH, ixt)
src=image_list[0]
src_basename = os.path.basename(src).split('.'+ixt)[0]
dst_basename = DSTPATH+'/'+src_basename
print(dst_basename)
L0 = dst_basename+'_lev0.cub'        
L1 = dst_basename+'_lev1.cub'                
L1echo = dst_basename+'_lev1echo.cub'
L2 = dst_basename+'_lev2.cub'
L2std = dst_basename+'_lev2.'+oxt

/home/jovyan/Data/NAC//m183533265re


In [19]:
RAWtoL0(lronac2isis, src, L0)

In [29]:
import subprocess
try:
    lronaccal(L0,to=L1)
except subprocess.CalledProcessError as err:
                                    print('Had an ISIS error:')
                                    print(' '.join(err.cmd))
                                    print(err.stdout)
                                    print(err.stderr)
                                    raise err

Had an ISIS error:
lronaccal from=/home/jovyan/Data/NAC//m183533265re_lev0.cub to=/home/jovyan/Data/NAC//m183533265re_lev1.cub

**ERROR** No existing files found with a numerical version matching [NACR_MaskedPixels.????.pvl] in [/isis-data/lro/calibration].



CalledProcessError: Command '['lronaccal', 'from=/home/jovyan/Data/NAC//m183533265re_lev0.cub', 'to=/home/jovyan/Data/NAC//m183533265re_lev1.cub']' returned non-zero exit status 1.